# Consistent routings with output to compare performance




- Identical routing setups are rarely reused across multiple plants, indicating low process standardization.

In [0]:
%sql
--Join consistent routings with output to compare performance
WITH consistent_routings AS (
    SELECT
        material_id,
        setup_time_mins,
        cycle_time_sec,
        labour_time_sec
    FROM workspace.default.routings
    GROUP BY material_id, setup_time_mins, cycle_time_sec, labour_time_sec
    HAVING COUNT(DISTINCT plant_id) > 1
)

SELECT
    o.material,
    o.production_order,
    o.plant,
    r.setup_time_mins,
    r.cycle_time_sec,
    r.labour_time_sec,
    ROUND(SUM(o.approved_quantity) / SUM(o.actual_net_production_time / 3600), 2) AS avg_output_per_hour
FROM workspace.default.output AS o
JOIN workspace.default.routings AS r
    ON o.production_order = r.order_id
    AND o.material = r.material_id
    AND o.plant = r.plant_id
    AND o.line = r.line
JOIN consistent_routings cr
    ON r.material_id = cr.material_id
    AND r.setup_time_mins = cr.setup_time_mins
    AND r.cycle_time_sec = cr.cycle_time_sec
    AND r.labour_time_sec = cr.labour_time_sec
GROUP BY o.material, o.production_order, o.plant, r.setup_time_mins, r.cycle_time_sec, r.labour_time_sec
ORDER BY o.material, avg_output_per_hour DESC;


material,production_order,plant,setup_time_mins,cycle_time_sec,labour_time_sec,avg_output_per_hour
4845494,303748,2,59,12.5,12.5,3621.69
5788376,335502,3,46,9.863,9.863,588.99
6175701,303732,2,55,12.522,12.522,3834.91
7807075,341286,3,28,6.545,6.545,3446.92
